# Pandas API on Spark

## Przygotowania

Na początku pobierz dane, które będziemy przetwarzali 

In [ ]:
%%sh
wget https://jankiewicz.pl/bigdata/bigdata-sp/netflix.zip

Następnie je rozpakuj 

In [ ]:
%%sh
unzip netflix.zip

Załaduj je do systemu plików HDFS

In [ ]:
%%sh
hadoop fs -put movie_titles.csv /tmp/
hadoop fs -put rates /tmp/

Sprawdź czy są dostępne w ich źródłowych (dla naszego przetwarzania) miejscach 

In [ ]:
%%sh
hadoop fs -ls /tmp

Podglądnij jak wygląda ich budowa 

In [ ]:
%%sh
hadoop fs -head /tmp/movie_titles.csv

In [ ]:
%%sh
hadoop fs -head /tmp/rates/part-00.csv

## Odczyt danych

Na początek kontekst

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Ustawmy stosowne zmienne środowiskowe, a następnie zaimportujmy potrzebne biblioteki

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [5]:
import pyspark.pandas as ps

Zaczytajmy dane do *Pandas Dataframe on Spark*

Na początku informacje na temat filmów

In [16]:
movies_ps = ps.read_csv('netflix/movie_titles.csv')

C:\src\spark-3.3.3-bin-hadoop3\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Dowiedzmy się jak wygląda "budowa" naszych danych

In [17]:
movies_ps.shape

(17770, 3)

In [18]:
movies_ps.columns

Index(['ID', 'Year', 'Title'], dtype='object')

In [19]:
movies_ps.head()

,ID,Year,Title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


Czas zatem na oceny

In [20]:
rates_ps = ps.read_csv('netflix/rates')

C:\src\spark-3.3.3-bin-hadoop3\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Dowiedzmy się jak wygląda "budowa" naszych danych

In [12]:
rates_ps.shape

(14780972, 2)

In [13]:
rates_ps.columns

Index(['date', 'rate'], dtype='object')

In [14]:
rates_ps.head()

TypeError: Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.

TypeError: Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.

To co, jesteśmy w domu? Zatem do dzieła. Trzy proste zadania

# Zadania

## Zadanie 1

Określ ile filmów nie uzyskało żadnej oceny. Utwórz *Pandas Dataframe on Spark*, który będzie zawierał komplet informacji filmów ograniczonych tylko do tych, które mają chociaż jedną ocenę.  

In [ ]:
movies_ps.columns

In [21]:
movies_ps['ID'].nunique()

17770

In [ ]:
ps.merge(movies_ps, rates_ps, left_on='ID', right_on='film_id')['ID'].nunique()

## Zadanie 2

Dla każdej dekady premiery filmów utwórz dwa rankingi

- trzech filmów o największej liczbie ocen
- trzech filmów o największej średniej ocen

In [22]:
ps.merge(movies_ps, rates_ps, left_on='ID', right_on='film_id').groupby(by=["Year"] // 10).nunique()

TypeError: unsupported operand type(s) for //: 'list' and 'int'

## Zadanie 3

Dla trzech wybranych użytkowników, wśród tych, którzy ocenili co najemniej 10 filmów, wyznacz trzy najbardziej rekomendowane filmy do obejrzenia. 

Wskaż źródła opisujące zastosowane przez Ciebie podejście. 

Koniecznie uzupełnij kod szczegółowymi komentarzami. 